In [2]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
from bs4 import BeautifulSoup

from function.add_crawlling import NewsCrawler

In [3]:
def remove_specific_date_format(date_str):
    # 'YY:MM:DD' 형식의 날짜 패턴 제거 (여기서는 예시로 'HH:MM:SS' 형식을 고려)
    pattern = r'\d{2}:\d{2}:\d{2}'
    # 해당 패턴에 일치하는 모든 부분을 공백으로 대체
    new_date_str = re.sub(pattern, '', date_str)
    return new_date_str.strip()

def reformat_date_to_yyyymmdd(date_str):
    # 다양한 날짜 형식에 대한 정규 표현식
    pattern = r'(\d{2,4})[년.-]?\s*(\d{1,2})[월.-]?\s*(\d{1,2})일?'
    match = re.search(pattern, date_str)
    
    if match:
        year, month, day = match.groups()
        # 년도가 2자리인 경우 처리 (예: '19' -> '2019')
        if len(year) == 2:
            year = '20' + year
        # 월과 일을 두 자리 숫자로 변환
        month = month.zfill(2)
        day = day.zfill(2)
        return f"{year}{month}{day}"
    else:
        return "날짜 없음"

def remove_first_date(date_str):
    # 날짜 패턴에 해당하는 첫 번째 매치를 제거, 이후 다시 패턴 추출(last_Date)
    pattern = r'(\d{2,4})[년.-]?\s*(\d{1,2})[월.-]?\s*(\d{1,2})일?'
    new_date_str = re.sub(pattern, '', date_str, count=1)  # count=1은 첫 번째 매치만 대체
    return new_date_str.strip()  # 공백 제거

def date_process(df, column='Date'):
    df[column] = df[column].apply(remove_specific_date_format)
    df[f'first_{column}'] = df[column].apply(reformat_date_to_yyyymmdd)
    df[f'last_{column}'] = df[column].apply(remove_first_date)
    df[f'last_{column}'] = df[f'last_{column}'].apply(reformat_date_to_yyyymmdd)
    return df

In [4]:
# 네이버 검색 페이지
def go_naver(driver):
    url = 'https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query='
    driver.get(url)


# go_naver(driver)

In [5]:
def click_option(driver, type='지면기사', service_type='PC'):
    # 옵션 버튼 클릭
    option_button = driver.find_element(By.CLASS_NAME, "btn_option._search_option_open_btn")
    option_button.click()
    time.sleep(0.1)

    # 카테고리 옵션을 위한 매핑
    type_mapping = {
        '전체': 0,
        '포토': 1,
        '동영상': 2,
        '지면기사': 3,
        '보도자료': 4,
        '자동생성기사': 5
    }
    # 서비스 타입 옵션을 위한 매핑
    service_mapping = {
        '전체': ('serviceall', 0),
        '모바일': ('servicem', 1),
        'PC': ('servicep', 2)
    }

    option_value = type_mapping.get(type, 3) # 기본값 '지면기사'
    service_value, service_index = service_mapping.get(service_type, ('serviceall', 0)) # 기본값 '전체'

    # 카테고리 옵션 선택 (type에 따라)
    if option_value != 0:
        type_button_selector = f'div[role="tablist"] a[onclick^="news_submit_category_option({option_value});"]'
        type_button = driver.find_element(By.CSS_SELECTOR, type_button_selector)
        type_button.click()

    time.sleep(0.1)
    
    # 서비스 타입 옵션 선택 (service_type에 따라)
    if service_index != 0:
        service_button_selector = f'div[role="tablist"] a[onclick^="news_submit_service_option({service_index}, \'{service_value}\');"]'
        service_button = driver.find_element(By.CSS_SELECTOR, service_button_selector)
        service_button.click()

    time.sleep(0.1)
    # 직접 입력 버튼 클릭
    직접입력_button = driver.find_element(By.CSS_SELECTOR, "a.txt.txt_option._calendar_select_trigger")
    직접입력_button.click()

# click_option(driver, type='동영상', service_type='전체')

In [6]:
def set_start(driver, start_date=20150101):
    
    start_date = str(start_date)
    start_date_trigger = driver.find_element(By.CSS_SELECTOR, "a._start_trigger")
    start_date_trigger.click()
    
    year_to_select = start_date[0:4]
    year_element = driver.find_element(By.XPATH, f"//li[@data-value='{year_to_select}']/a")
    year_element.click()
    
    month_to_select = str(int(start_date[4:6]))
    month_element = driver.find_element(By.XPATH, "//strong[contains(text(), '월(Month)')]/following-sibling::div//li[@data-value='" + month_to_select + "']/a")
    month_element.click()
    
    day_to_select = str(int(start_date[6:8]))
    day_element = driver.find_element(By.XPATH, "//strong[contains(text(), '일(Day)')]/following-sibling::div//li[@data-value='" + day_to_select + "']/a")
    day_element.click()

# set_start(driver)

In [7]:
def set_end(driver, end_date=None):
    if int(end_date) < 20160131:
        end_date = 20160131
        
    end_date = str(end_date)
    end_date_trigger = driver.find_element(By.CSS_SELECTOR, "a._end_trigger")
    end_date_trigger.click()

    year_to_select = end_date[0:4]
    year_element = driver.find_element(By.XPATH, f"//li[@data-value='{year_to_select}']/a")
    year_element.click()

    month_to_select = str(int(end_date[4:6]))
    month_element = driver.find_element(By.XPATH, "//strong[contains(text(), '월(Month)')]/following-sibling::div//li[@data-value='" + month_to_select + "']/a")
    month_element.click()

    day_to_select = str(int(end_date[6:8]))
    day_element = driver.find_element(By.XPATH, "//strong[contains(text(), '일(Day)')]/following-sibling::div//li[@data-value='" + day_to_select + "']/a")
    day_element.click()


# end_date = '20210426'
# set_end(driver, end_date)

In [8]:
def click_application(driver):
    application_button = driver.find_element(By.CLASS_NAME, "btn_apply")
    application_button.click()

# click_application(driver)

In [9]:
def serch_keyword(driver, data, index=0, API_name='keyword_API'):
    search_query = data.at[index, API_name]

    search_field = driver.find_element(By.NAME, "query")
    search_field.clear()
    search_field.send_keys(search_query)

    search_button = driver.find_element(By.CLASS_NAME, "bt_search")
    search_button.click()

# serch_keyword(driver, data, 99, 'keyword_API')

In [10]:
def crawl_news_data(driver, n=3):
    current_url = driver.current_url

    # 현재 URL에서 HTML 콘텐츠 가져오기
    response = requests.get(current_url)
    html_content = response.text

    # BeautifulSoup를 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')

    # 뉴스 데이터를 저장할 리스트 초기화
    news_data = []

    # 모든 <li> 요소를 찾고 클래스 "bx"가 있는 뉴스 아이템 처리
    for news_item in soup.find_all('li', class_='bx')[7:]:
        # 언론사 이름 추출
        press = news_item.find('a', class_='info press')
        press_name = press.get_text(strip=True) if press else 'No press name'

        # 뉴스 제목과 URL 추출
        title_element = news_item.find('a', class_='news_tit')
        title = title_element.get('title', 'No title').strip() if title_element else 'No title'
        url = title_element.get('href', 'No URL').strip() if title_element else 'No URL'
        domain_raw = urlparse(url).netloc
        domain = domain_raw.replace('.', '_')

        # 날짜가 포함된 <span class="info"> 찾기
        info_spans = news_item.find_all('span', class_='info')
        date_text = None
        for span in info_spans:
            if not span.find('i', class_='spnew ico_paper'):
                date_text = span.get_text(strip=True)
                break
        date_text = date_text if date_text else 'No date'

        # 추출한 데이터를 리스트에 추가
        news_data.append({
            'Press': press_name,
            'Title': title,
            'URL': url,
            'Domain': domain,  # 도메인 추출 수정
            'Date': date_text
        })

    # 데이터 프레임 생성
    return pd.DataFrame(news_data[0:n])

# news_df = crawl_news_data(driver, n=10)
# news_df

In [11]:
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup
import requests

def crawl_news_data1(driver, n=3):
    current_url = driver.current_url

    # 현재 URL에서 HTML 콘텐츠 가져오기
    response = requests.get(current_url)
    html_content = response.text

    # BeautifulSoup를 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')

    # 뉴스 데이터를 저장할 리스트 초기화
    news_data = []

    # 수집을 원치 않는 도메인 리스트
    exclude_domains = not_in_content

    # 모든 <li> 요소를 찾고 클래스 "bx"가 있는 뉴스 아이템 처리
    count = 0  # 수집된 기사 수 카운트
    for news_item in soup.find_all('li', class_='bx')[7:]:
        if count >= n:
            break

        # 뉴스 제목과 URL 추출
        title_element = news_item.find('a', class_='news_tit')
        title = title_element.get('title', 'No title').strip() if title_element else 'No title'
        url = title_element.get('href', 'No URL').strip() if title_element else 'No URL'
        domain_raw = urlparse(url).netloc
        domain = domain_raw.replace('.', '_')

        # 해당 도메인이 제외 리스트에 있으면 건너뛰기
        if any(excluded in domain for excluded in exclude_domains):
            print(f'해당 도메인이 존재하지 않습니다. domain: {domain}')
            continue

        # 언론사 이름 추출
        press = news_item.find('a', class_='info press')
        press_name = press.get_text(strip=True) if press else 'No press name'

        # 날짜가 포함된 <span class="info"> 찾기
        info_spans = news_item.find_all('span', class_='info')
        date_text = None
        for span in info_spans:
            if not span.find('i', class_='spnew ico_paper'):
                date_text = span.get_text(strip=True)
                break
        date_text = date_text if date_text else 'No date'

        # 추출한 데이터를 리스트에 추가
        news_data.append({
            'Press': press_name,
            'Title': title,
            'URL': url,
            'Domain': domain,
            'Date': date_text
        })
        count += 1  # 수집된 기사 수 업데이트

    # 데이터 프레임 생성
    return pd.DataFrame(news_data)


# 관련 기사 추가 시작

In [12]:
data_path = './data/'
data_name = 'KB_3_0.8.csv'

data = pd.read_csv(data_path + data_name)
print(len(data))

condition = data['Body'].isna() | (data['Body'].str.strip() == '')
data = data.loc[~condition]
data = data.reset_index(drop=True)
print(len(data))

1352
1352


In [13]:
############ 파라미터 ###############
n = 3  # 추가할 관련 기사의 갯수
indicator_date = '마지막'  # 시간 반영 여부, 처음 or 마지막 or 없음
##################################### 


### 날짜 데이터 처리

In [14]:
condition1 = data['Date'].isna()
condition2 = data['Date'].str.strip() == ''
data[condition1 | condition2]

,ID,Title,Source Title,Source Link,Final Registration,Modification Reason,Domain,Date,Head,Body,Useful,Fact,keyword
1294,Q59,"전기요금 검침 날짜를 변경하면, 올해 '요금 폭탄' 안 맞는다?",링크(제목없음),http://www.ftc.go.kr/www/selectReportUserView....,2018.08.09 10:34,NaN,www_ftc_go_kr,NaN,한국전력공사의 기본공급약관상 불공정약관조항 시정,공정거래위원회는 한국전력공사(이하 한전)가 고객의 동의없이 일방적으로 검침일을 정하...,보도자료,0,"전기이용, 공정거래위원회는, 전력사용"


In [15]:
condition = data['Title'] == '전기요금 검침 날짜를 변경하면, 올해 \'요금 폭탄\' 안 맞는다?'
data.loc[condition, 'Date'] = '2018년 8월 7일(화)'
data[condition]

,ID,Title,Source Title,Source Link,Final Registration,Modification Reason,Domain,Date,Head,Body,Useful,Fact,keyword
1294,Q59,"전기요금 검침 날짜를 변경하면, 올해 '요금 폭탄' 안 맞는다?",링크(제목없음),http://www.ftc.go.kr/www/selectReportUserView....,2018.08.09 10:34,NaN,www_ftc_go_kr,2018년 8월 7일(화),한국전력공사의 기본공급약관상 불공정약관조항 시정,공정거래위원회는 한국전력공사(이하 한전)가 고객의 동의없이 일방적으로 검침일을 정하...,보도자료,0,"전기이용, 공정거래위원회는, 전력사용"


In [16]:
data = date_process(data, 'Date')

# 변환되지 않은 날짜 데이터 조회
condition = data['first_Date'] == '날짜 없음'
unconverted_dates = data.loc[condition, 'Date']

if len(unconverted_dates) < 1:
    print('모두 추출되었습니다.')

display(data[['Date', 'first_Date', 'last_Date']].head(4))

모두 추출되었습니다.


,Date,first_Date,last_Date
0,입력2023.09.27 05:00,20230927,날짜 없음
1,입력 : 2023.06.13 17:04수정 : 2023.06.13 18:27,20230613,20230613
2,2024.01.29 16:54,20240129,날짜 없음
3,민생희망본부주거2024-01-17598,20240117,날짜 없음


## 관련 기사 반영

## 제외할 domain 설정

In [17]:
with open('./data/domain_list.txt', 'r', encoding='utf-8') as file:
    content = file.read()

domain = pd.read_csv('domain_view.csv', encoding='utf-8-sig')
not_in_content = [domain for domain in domain['Domain'] if domain not in content]
not_in_content

['news_joins_com',
 'news_imaeil_com',
 'mydaily_co_kr',
 'www_dgmbc_com',
 'www_chiefexe_com']

## 기본 크롤링 함수 설정

In [18]:
def base_crawlling(data, index):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-features=SameSiteByDefaultCookies,CookiesWithoutSameSiteMustBeSecure")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    time.sleep(1)
    print(f'Row {index} 시작, 관련기사: {n}개 추가')

    try:
        go_naver(driver)
        time.sleep(1)
        
        # click_option(driver, type='지면기사', service_type='모바일')
        click_option(driver, type='전체', service_type='전체')
        time.sleep(0.1)
        
        set_start(driver)
        
        if indicator_date == '마지막':
            if data.at[index, 'last_Date'] == '날짜 없음':
                end_date = data.at[index, 'first_Date']
            else:
                end_date = data.at[index, 'last_Date']
            set_end(driver, end_date)
        elif indicator_date == '처음':
            end_date = data.at[index, 'first_Date']
            set_end(driver, end_date)
        elif indicator_date == '없음':
            pass
        
        click_application(driver)
        
        serch_keyword(driver, data, index, 'keyword')  # data의 index, 키워드 추출 패키지의 종류가 들어감
        time.sleep(3)
        new_df = crawl_news_data1(driver, n)
        
        # NewsCrawler 호출을 통해 새로운 크롬 실행
        crawler = NewsCrawler()
        time.sleep(0.5)

        max_retries = 2
        attempt = 0  # 현재 시도 횟수를 추적
        success = False  # 작업 성공 여부를 확인

        for j in range(len(new_df)):
            url = new_df.at[j, 'URL']
            # print('접속 url: ', url,)
            site_key = new_df.at[j, 'Domain']
            
            max_retries = 3
            attempt = 0  # 현재 시도 횟수를 추적
            success = False  # 작업 성공 여부를 확인
            
            while attempt < max_retries and not success:
                print(f'Row {index}-{j} 시작, 시도: {attempt+1}')
                try:
                    date, title, body = crawler.crawl(url, site_key)
                    data.loc[index, f'Date_{j}'] = date
                    data.loc[index, f'Head_{j}'] = title
                    data.loc[index, f'Body_{j}'] = body
                    
                    print(f'Row {index}-{j} 성공')
                    success = True
                except Exception as e:
                    print(f'Row {index}-{j} 실패, {site_key}:{url}')
                    print(f'에러메세지, {e}')
                    data.loc[index, f'Date_{j}'] = site_key
                    data.loc[index, f'Head_{j}'] = url
                    data.loc[index, f'Body_{j}'] = '실패'
                    attempt += 1

    except Exception as e:
        print(f'Row {index} 실패')
        time.sleep(0.5)
    finally:
        print(f'Row {index} 종료\n')
        time.sleep(0.5)
        crawler.close()
        driver.quit()
        
# index = 10  # data의 index
# base_crawlling(data, index)

In [18]:
data.shape

(1352, 15)

## 병렬처리를 통해 실행

In [19]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm

start_time = time.time()
indexes = range(len(data)) 
print(indexes)

# ThreadPoolExecutor를 사용하여 병렬 실행
with ThreadPoolExecutor(max_workers=12) as executor:
    # max_workers는 동시에 실행할 작업의 수입니다. 시스템에 따라 조정해주세요.
    futures = {executor.submit(base_crawlling, data, index): index for index in indexes}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing", unit="task"):
        index = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"{index}에서 예외 발생 {e}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"코드 실행 시간: {elapsed_time}초")

range(0, 1352)
Row 3 시작, 관련기사: 3개 추가
Row 6 시작, 관련기사: 3개 추가
Row 2 시작, 관련기사: 3개 추가
Row 5 시작, 관련기사: 3개 추가
Row 1 시작, 관련기사: 3개 추가
Row 7 시작, 관련기사: 3개 추가
Row 9 시작, 관련기사: 3개 추가
Row 0 시작, 관련기사: 3개 추가
Row 10 시작, 관련기사: 3개 추가
Row 11 시작, 관련기사: 3개 추가
Row 8 시작, 관련기사: 3개 추가
Row 4 시작, 관련기사: 3개 추가
Row 3-0 시작, 시도: 1
Row 2-0 시작, 시도: 1
Row 5-0 시작, 시도: 1
Row 7-0 시작, 시도: 1
Row 6-0 시작, 시도: 1
Row 1-0 시작, 시도: 1
Row 8-0 시작, 시도: 1
Row 4-0 시작, 시도: 1
Row 11 종료

Row 4-0 성공
Row 4-1 시작, 시도: 1
Row 1-0 성공
Row 1-1 시작, 시도: 1
Row 7-0 성공
Row 7-1 시작, 시도: 1
Row 4-1 실패, www_news1_kr:https://www.news1.kr/photos/view/?6013574
에러메세지, 'NoneType' object has no attribute 'find'
Row 4-1 시작, 시도: 2
Row 12 시작, 관련기사: 3개 추가
Row 4-1 실패, www_news1_kr:https://www.news1.kr/photos/view/?6013574
에러메세지, 'NoneType' object has no attribute 'find'
Row 4-1 시작, 시도: 3
Row 4-1 실패, www_news1_kr:https://www.news1.kr/photos/view/?6013574
에러메세지, 'NoneType' object has no attribute 'find'
Row 4-2 시작, 시도: 1
Row 2-0 성공
Row 2-1 시작, 시도: 1
Row 1-1 성공
Row 1-2 시작, 

## 수집되지 않은 데이터 확인

In [49]:
condition1 = (data['Body_0'].isna()) | (data['Body_0']=='실패') | (data['Body_0'].str.strip() == '')
condition2 = (data['Body_1'].isna()) | (data['Body_1']=='실패') | (data['Body_1'].str.strip() == '')
condition3 = (data['Body_2'].isna()) | (data['Body_2']=='실패') | (data['Body_2'].str.strip() == '')

data.loc[condition1, ['keyword', 'Body_0', 'Body_1', 'Body_2']]

KeyError: 'Body_0'

In [37]:
data.at[1, 'Body_1']

('[서울=뉴스핌] 송현도 기자 =\xa04월16일의약속국민연대(416연대)가 세월호참사 10주기 시민 기억식을 열고 희생자들을 추모했다. 416연대는 "생명이 존중받는 안전한 사회를 함께 만들어 주실 것을 부탁한다"며 정부에 정보 공개와 추가 조사를 촉구했다.  416연대는 16일 오후 4시16분 서울 중구 서울시의회 앞 세월호 기억공간 \'기억과 빛\'에서 세월호 10주기 시민 기억식을 열었다.   이날 기억제에는 세월호를 잊지 않은 시민들과 유가족의 발걸음이 이어졌다. 기억 공간 앞에는 100여명이 넘는 시민들이 운집해 묵념과 함께 기억제를 같이 했다.  기억제에 앞서 시민들은 헌화를 위해 길게 줄을 서 기다리고 기억 공간에 놓인 사진과 세월호 모형을 지긋이 바라보기도 했다.  헌화에 참여한 대학생 김모(21) 씨는 "6촌 오빠가 단원고에 다니던 세월호 희생자 중 한 명"이라며 "사고 당시에는 초등학생이라 어려서 희생자인 것을 몰랐다가 고등학교 2학년이 되어서야 알았다"며 "그것 때문에 더 죄책감이 커서 매년 추모 행사에 나오고 있다"고 밝혔다.  이어 "매년 많은 사람이 올거란 기대는 섣불리 하지 않는데도 (막상 기억제에 오니) 꽃도 많이 쌓여있어 다들 기억해 주려 노력하는구나 싶어 감사하다"며 "다시는 이런 일이 발생하면 안 된다고 생각하고 2차, 3차 가해가 반복되지 않았으면 좋겠다"고 덧붙였다.   이날 발언대에 올라온 서울시의회 박수빈 의원은 세월호 관련 서적을 집필하면서 생존자 진술과 영상 자료를 많이 봤다고 회고했다. 박 시의원은 "당시 세월호에 타고 있던 아이들(단원고 학생들)이 5살이었던 어린아이를 구하기 위해서 너도나도 \'여기 아기 있어요\'라고 외쳤고 그 아이가 구조됐다. 이제 그 아이는 단원고 아이들의 나이"라고 했다.  이어 "그 10년 전의 하루의 기억이 아마 오늘날 우리를 만들었을 거라고 생각한다"며 "다시는 이런 일이 발생해선 안 된다는 다짐을 현실로 만드는 것이 세월호를 기억하는 길"이라고 말했다.  기억 공간 활동가 유혜

In [ ]:
data.to_csv('./data/KB_3_0.8_related_news_time.csv', index=False, encoding='utf-8-sig')

In [15]:
chrome_options = Options()
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--ssl-protocol=any")
chrome_options.add_argument("--ignore-ssl-errors=true")
chrome_options.add_argument("--headless")  # 헤드리스 모드 활성화
chrome_options.add_argument("--disable-features=SameSiteByDefaultCookies,CookiesWithoutSameSiteMustBeSecure")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [64]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [17]:
import pandas as pd
import time

# 모든 관련 기사 데이터를 저장할 빈 데이터프레임 초기화
all_news_df = pd.DataFrame()

# 데이터프레임의 길이만큼 반복
for index in range(len(data)):
    try:
        go_naver(driver)
        time.sleep(1)

        click_option(driver, type='전체', service_type='전체')
        time.sleep(0.1)

        set_start(driver)

        # 'last_Date'가 '날짜 없음'인지 확인
        if data.at[index, 'last_Date'] == '날짜 없음':
            end_date = data.at[index, 'first_Date']
        else:
            end_date = data.at[index, 'last_Date']

        set_end(driver, end_date)

        # 검색 키워드 설정 및 검색 실행
        serch_keyword(driver, data, index, 'keyword')  # 오타 주의: search_keyword가 맞을 수 있음

        # 관련 기사 데이터 수집
        news_df = crawl_news_data(driver, n=10)

        # 수집된 기사 데이터를 all_news_df에 추가
        all_news_df = pd.concat([all_news_df, news_df], ignore_index=True)

    except Exception as e:
        print(f"Error processing index {index}: {e}")
        time.sleep(0.5)



Error processing index 131: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//strong[contains(text(), '월(Month)')]/following-sibling::div//li[@data-value='6']/a"}
  (Session info: chrome-headless-shell=123.0.6312.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00C44CA3+225091]
	(No symbol) [0x00B74DF1]
	(No symbol) [0x00A19A7A]
	(No symbol) [0x00A5175B]
	(No symbol) [0x00A5188B]
	(No symbol) [0x00A87882]
	(No symbol) [0x00A6F5A4]
	(No symbol) [0x00A85CB0]
	(No symbol) [0x00A6F2F6]
	(No symbol) [0x00A479B9]
	(No symbol) [0x00A4879D]
	sqlite3_dbdata_init [0x010B9A43+4064547]
	sqlite3_dbdata_init [0x010C104A+4094762]
	sqlite3_dbdata_init [0x010BB948+4072488]
	sqlite3_dbdata_init [0x00DBC9A9+930953]
	(No symbol) [0x00B807C4]
	(No symbol) [0x00B7ACE8]
	(No symbol) [0x00B7AE11]
	(No symbol) [0x00B6CA80]
	BaseThreadInitThunk

In [18]:
all_news_df

,Press,Title,URL,Domain,Date
0,중앙일보,"국민 4명중 1명 ""경증으로 응급실""…89%는 ""이용 제한땐 동참""",https://www.joongang.co.kr/article/25195924,www_joongang_co_kr,2023.09.27.
1,메디컬월드뉴스,"[9월 5일 병원계 이모저모②]국립암센터, 고려대의료원, 이대목동, 해운대백병원 등 소식",https://medicalworldnews.co.kr/news/view.php?i...,medicalworldnews_co_kr,2023.09.05.
2,의약뉴스,"삼육서울병원, 우수검사실 신임 인증 획득 外​​​​​",http://www.newsmp.com/news/articleView.html?id...,www_newsmp_com,3주 전
3,뉴스1언론사 선정,"""떡이 목에 걸려 숨을 못 쉬어요""…명절 응급상황 발생시 대처요령",https://www.news1.kr/articles/5185269,www_news1_kr,2023.09.29.
4,의약뉴스,아산상 대상에 헤브론의료원 김우정 의료원장 선정 外,http://www.newsmp.com/news/articleView.html?id...,www_newsmp_com,2021.10.20.
...,...,...,...,...,...
7740,국민일보,폭행·취객 행패 시달려도 “달래는 게 전부”라는 상가 경비원,https://www.kmib.co.kr/article/view.asp?arcid=...,www_kmib_co_kr,1주 전
7741,브릿지경제,"오세훈표 자율주행버스, 전국으로 확산",https://www.viva100.com/main/view.php?key=2024...,www_viva100_com,1주 전
7742,아주경제,"서울시, '오세훈표 자율주행버스' 민생맞춤 우수사례로 전국 확산…7월부터는 기후동행...",https://www.ajunews.com/view/20240404102336847,www_ajunews_com,1주 전
7743,엠투데이,"서울이 시작한 자율주행버스, 이제 전국으로 확산",https://www.autodaily.co.kr/news/articleView.h...,www_autodaily_co_kr,1주 전


In [20]:
all_news_df.to_csv('domain_test.csv', index=False, encoding='utf-8-sig')

In [19]:
# Domain 열에 대한 빈도수 계산
domain_freq_series = all_news_df['Domain'].value_counts()

# 빈도수 시리즈를 데이터프레임으로 변환
domain_freq_df = domain_freq_series.reset_index()
domain_freq_df.columns = ['Domain', 'Frequency']

# 결과 확인
domain_freq_df

# 필요한 경우 CSV 파일로 저장
domain_freq_df.to_csv('domain_view.csv', index=False, encoding='utf-8-sig')